# LSTM Preprocess

In [61]:
import pandas as pd

#Reading the data csv
ticker = "AAPL"
df_price = pd.read_csv(f"{ticker}_Daily_Data.csv")

# Convert the 'Date' column to datetime if it's not already
df_price['Date'] = pd.to_datetime(df_price['Date'])

# Specify the date from which you want to keep the rows
start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date

# Filter the DataFrame to keep rows from the start_date onwards
df_price = df_price[df_price['Date'] >= start_date]

# Reset the index of the filtered DataFrame
df_price.reset_index(drop=True, inplace=True)

df_price.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
395,2023-09-25,174.199997,176.970001,174.149994,176.080002,176.080002,46172700
396,2023-09-26,174.820007,175.199997,171.660004,171.960007,171.960007,64588900
397,2023-09-27,172.619995,173.039993,169.050003,170.429993,170.429993,66830700
398,2023-09-28,169.339996,172.029999,167.619995,170.690002,170.690002,56294400
399,2023-09-29,172.020004,173.070007,170.339996,171.210007,171.210007,51814200


In [62]:
df_news = pd.read_csv(f"{ticker}_News_Content.csv")

# Filter rows with relevance_score > 0.5 and positive ticker_sentiment_score
# df_news = df_news[(df_news['relevance_score'] > 0.5)]
# & (df_news['ticker_sentiment_score'] > 0)]

# Extract date components
df_news['Date'] = pd.to_datetime(df_news['time_published'], format='%Y%m%dT%H%M%S').dt.date
df_news['Time'] = pd.to_datetime(df_news['time_published'], format='%Y%m%dT%H%M%S').dt.time

# Convert date to datetime format
df_news['Date'] = pd.to_datetime(df_news['Date'])

# Define a custom function to determine if a given date is a weekend
def is_weekend(date):
    return date.weekday() >= 5  # 5 and 6 represent Saturday and Sunday

# Create a new column 'Date_adjusted' that shifts weekend dates to the next Monday
df_news['Date'] = df_news['Date'].apply(lambda x: x + pd.DateOffset(days=2) if is_weekend(x) else x)

# Drop unnecessary columns
columns_to_drop = ['ticker', 'time_published', 'Time', 'overall_sentiment_label', 'ticker_sentiment_label']
df_news.drop(columns=columns_to_drop, inplace=True)

# Group by 'Date_adjusted' and concatenate titles and summaries into a single paragraph
agg_functions = {
    'title': ' '.join,  # Concatenate titles
    'summary': ' '.join,  # Concatenate summaries
    'overall_sentiment_score': 'mean',  # Calculate the mean
    'relevance_score': 'mean',  # Calculate the mean
    'ticker_sentiment_score': 'mean'  # Calculate the mean
}

df_news = df_news.groupby('Date').agg(agg_functions).reset_index()

# Rename columns if needed
df_news.rename(columns={'overall_sentiment_score': 'average_overall_sentiment_score',
                          'relevance_score': 'average_relevance_score',
                          'ticker_sentiment_score': 'average_ticker_sentiment_score'},
                 inplace=True)

# Define a custom function to apply the condition
def calculate_sentiment(score):
    if score > 0.5:
        return 1
    else:
        return 0

# Apply the custom function to create the 'sentiment score' column
df_news['sentiment_score'] = df_news['average_overall_sentiment_score'].apply(calculate_sentiment)

df_news.tail()

,Date,title,summary,average_overall_sentiment_score,average_relevance_score,average_ticker_sentiment_score,sentiment_score
404,2023-09-22,Apple Cuts Back On Retail Employee Raises As P...,Apple Inc. AAPL has reportedly offered smaller...,0.169165,0.498261,0.317322,0
405,2023-09-25,"Buy the Dip in Apple, or Has the Trend Turned ...",Editor's note: Any and all references to time ...,0.172110,0.471907,0.223139,0
406,2023-09-26,3 Warren Buffett Stocks to See Skyrocketing AI...,Buffett's portfolio and AI -- an unexpected ma...,0.108096,0.412371,0.185967,0
407,2023-09-27,Is Apple Headed For Titanium Gate? Expert Iden...,"Apple Inc.'s AAPL latest smartphone series, th...",0.145487,0.353615,0.102633,0
408,2023-09-28,'Lost All Visibility' On The Apple Car Project...,"If you have been waiting for the Apple Car, yo...",0.187439,0.469908,0.183889,0


In [63]:
# Convert 'Date' column to datetime type in both dataframes
df_news['Date'] = pd.to_datetime(df_news['Date'])
df_price['Date'] = pd.to_datetime(df_price['Date'])

# Perform a left outer join based on the 'Date' column
merged_df = pd.merge(df_price, df_news, on='Date', how='left')

# Fill NaN values with 0
merged_df.fillna(0, inplace=True)

# Drop duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Display the merged dataframe
merged_df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,title,summary,average_overall_sentiment_score,average_relevance_score,average_ticker_sentiment_score,sentiment_score
394,2023-09-25,174.199997,176.970001,174.149994,176.080002,176.080002,46172700,"Buy the Dip in Apple, or Has the Trend Turned ...",Editor's note: Any and all references to time ...,0.172110,0.471907,0.223139,0.0
395,2023-09-26,174.820007,175.199997,171.660004,171.960007,171.960007,64588900,3 Warren Buffett Stocks to See Skyrocketing AI...,Buffett's portfolio and AI -- an unexpected ma...,0.108096,0.412371,0.185967,0.0
396,2023-09-27,172.619995,173.039993,169.050003,170.429993,170.429993,66830700,Is Apple Headed For Titanium Gate? Expert Iden...,"Apple Inc.'s AAPL latest smartphone series, th...",0.145487,0.353615,0.102633,0.0
397,2023-09-28,169.339996,172.029999,167.619995,170.690002,170.690002,56294400,'Lost All Visibility' On The Apple Car Project...,"If you have been waiting for the Apple Car, yo...",0.187439,0.469908,0.183889,0.0
398,2023-09-29,172.020004,173.070007,170.339996,171.210007,171.210007,51814200,0,0,0.000000,0.000000,0.000000,0.0


In [ ]:
import pandas as pd

#Reading the data csv
ticker = "AAPL"
df_price = pd.read_csv(f"{ticker}_Daily_Data.csv")
df_news = pd.read_csv(f"{ticker}_News_Content.csv")

# Convert the 'Date' column to datetime if it's not already
df_price['Date'] = pd.to_datetime(df_price['Date'])

# Specify the date from which you want to keep the rows
start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date

# Filter the DataFrame to keep rows from the start_date onwards
df_price = df_price[df_price['Date'] >= start_date]

# Reset the index of the filtered DataFrame
df_price.reset_index(drop=True, inplace=True)

# Filter rows with relevance_score > 0.5 and positive ticker_sentiment_score
# df_news = df_news[(df_news['relevance_score'] > 0.5)]
# & (df_news['ticker_sentiment_score'] > 0)]

# Extract date components
df_news['Date'] = pd.to_datetime(df_news['time_published'], format='%Y%m%dT%H%M%S').dt.date
df_news['Time'] = pd.to_datetime(df_news['time_published'], format='%Y%m%dT%H%M%S').dt.time

# Convert date to datetime format
df_news['Date'] = pd.to_datetime(df_news['Date'])

# Define a custom function to determine if a given date is a weekend
def is_weekend(date):
    return date.weekday() >= 5  # 5 and 6 represent Saturday and Sunday

# Create a new column 'Date_adjusted' that shifts weekend dates to the next Monday
df_news['Date'] = df_news['Date'].apply(lambda x: x + pd.DateOffset(days=2) if is_weekend(x) else x)

# Drop unnecessary columns
columns_to_drop = ['ticker', 'time_published', 'Time', 'overall_sentiment_label', 'ticker_sentiment_label']
df_news.drop(columns=columns_to_drop, inplace=True)

# Group by 'Date_adjusted' and concatenate titles and summaries into a single paragraph
agg_functions = {
    'title': ' '.join,  # Concatenate titles
    'summary': ' '.join,  # Concatenate summaries
    'overall_sentiment_score': 'mean',  # Calculate the mean
    'relevance_score': 'mean',  # Calculate the mean
    'ticker_sentiment_score': 'mean'  # Calculate the mean
}

df_news = df_news.groupby('Date').agg(agg_functions).reset_index()

# Rename columns if needed
df_news.rename(columns={'overall_sentiment_score': 'average_overall_sentiment_score',
                          'relevance_score': 'average_relevance_score',
                          'ticker_sentiment_score': 'average_ticker_sentiment_score'},
                 inplace=True)

# Define a custom function to apply the condition
def calculate_sentiment(score):
    if score > 0.5:
        return 1
    else:
        return 0

# Apply the custom function to create the 'sentiment score' column
df_news['sentiment_score'] = df_news['average_overall_sentiment_score'].apply(calculate_sentiment)

# Convert 'Date' column to datetime type in both dataframes
df_news['Date'] = pd.to_datetime(df_news['Date'])
df_price['Date'] = pd.to_datetime(df_price['Date'])

# Perform a left outer join based on the 'Date' column
merged_df = pd.merge(df_price, df_news, on='Date', how='left')

# Fill NaN values with 0
merged_df.fillna(0, inplace=True)

# Drop duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Display the merged dataframe
merged_df.tail()


# ARIMA and RL Agent Preprocess

In [5]:
import pandas as pd

#Reading the data csv
ticker = "AAPL"
df = pd.read_csv(f"{ticker}_Daily_Data.csv")

# Convert the 'Date' column to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Specify the date from which you want to keep the rows
start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date

# Filter the DataFrame to keep rows from the start_date onwards
filtered_df = df[df['Date'] >= start_date]

# Reset the index of the filtered DataFrame
filtered_df.reset_index(drop=True, inplace=True)

filtered_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-03-01,164.699997,166.600006,161.970001,163.199997,161.774796,83474400
1,2022-03-02,164.389999,167.360001,162.949997,166.559998,165.105453,79724800
2,2022-03-03,168.470001,168.910004,165.550003,166.229996,164.778336,76678400
3,2022-03-04,164.490005,165.550003,162.100006,163.169998,161.745056,83737200
4,2022-03-07,163.360001,165.020004,159.039993,159.300003,157.908875,96418800


In [8]:
import pandas as pd

def ARIMA_and_RL_Agent_Preprocess(tickers, start_date):
    filtered_dfs = {}  # Store filtered DataFrames for each ticker
    
    for ticker in tickers:
        # Read the data CSV for the current ticker
        df = pd.read_csv(f"{ticker}_Daily_Data.csv")

        # Convert the 'Date' column to datetime if it's not already
        df['Date'] = pd.to_datetime(df['Date'])

        # Filter the DataFrame to keep rows from the start_date onwards
        filtered_df = df[df['Date'] >= start_date]

        # Reset the index of the filtered DataFrame
        filtered_df.reset_index(drop=True, inplace=True)

        # Store the filtered DataFrame in the dictionary
        filtered_dfs[ticker] = filtered_df

    return filtered_dfs

# Example usage:
tickers = ["AAPL", "MSFT", "ORCL"]  # Replace with your list of tickers
start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date
filtered_data = ARIMA_and_RL_Agent_Preprocess(tickers, start_date)

# Access filtered DataFrames for specific tickers
aapl_filtered_df = filtered_data["AAPL"]
googl_filtered_df = filtered_data["MSFT"]
msft_filtered_df = filtered_data["ORCL"]

aapl_filtered_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-03-01,164.699997,166.600006,161.970001,163.199997,161.774796,83474400
1,2022-03-02,164.389999,167.360001,162.949997,166.559998,165.105453,79724800
2,2022-03-03,168.470001,168.910004,165.550003,166.229996,164.778336,76678400
3,2022-03-04,164.490005,165.550003,162.100006,163.169998,161.745056,83737200
4,2022-03-07,163.360001,165.020004,159.039993,159.300003,157.908875,96418800


In [7]:
import pandas as pd

# Method for data preprocessing
def preprocess_stock_data(ticker):
    # Reading the data csv
    df_price = pd.read_csv(f"{ticker}_Daily_Data.csv")
    df_news = pd.read_csv(f"{ticker}_News_Content.csv")

    # Convert the 'Date' column to datetime if it's not already
    df_price['Date'] = pd.to_datetime(df_price['Date'])

    # Specify the date from which you want to keep the rows
    start_date = pd.to_datetime('2022-03-01')  # Change this to your desired start date

    # Filter the DataFrame to keep rows from the start_date onwards
    df_price = df_price[df_price['Date'] >= start_date]

    # Reset the index of the filtered DataFrame
    df_price.reset_index(drop=True, inplace=True)

    # Extract date components
    df_news['Date'] = pd.to_datetime(df_news['time_published'], format='%Y%m%dT%H%M%S').dt.date

    # Convert date to datetime format
    df_news['Date'] = pd.to_datetime(df_news['Date'])

    # Define a custom function to determine if a given date is a weekend
    def is_weekend(date):
        return date.weekday() >= 5  # 5 and 6 represent Saturday and Sunday

    # Create a new column 'Date_adjusted' that shifts weekend dates to the next Monday
    df_news['Date'] = df_news['Date'].apply(lambda x: x + pd.DateOffset(days=2) if is_weekend(x) else x)

    # Drop unnecessary columns
    columns_to_drop = ['ticker', 'time_published', 'overall_sentiment_label', 'ticker_sentiment_label']
    df_news.drop(columns=columns_to_drop, inplace=True)

    # Group by 'Date_adjusted' and concatenate titles and summaries into a single paragraph
    agg_functions = {
        'title': ' '.join,  # Concatenate titles
        'summary': ' '.join,  # Concatenate summaries
        'overall_sentiment_score': 'mean',  # Calculate the mean
        'relevance_score': 'mean',  # Calculate the mean
        'ticker_sentiment_score': 'mean'  # Calculate the mean
    }

    df_news = df_news.groupby('Date').agg(agg_functions).reset_index()

    # Rename columns if needed
    df_news.rename(columns={'overall_sentiment_score': 'average_overall_sentiment_score',
                            'relevance_score': 'average_relevance_score',
                            'ticker_sentiment_score': 'average_ticker_sentiment_score'},
                   inplace=True)

    # Define a custom function to apply the condition
    def calculate_sentiment(score):
        if score > 0:
            return int(1)
        else:
            return int(0)

    # Apply the custom function to create the 'sentiment score' column
    df_news['sentiment_score'] = df_news['average_overall_sentiment_score'].apply(calculate_sentiment)

    # Convert 'Date' column to datetime type in both dataframes
    df_news['Date'] = pd.to_datetime(df_news['Date'])
    df_price['Date'] = pd.to_datetime(df_price['Date'])

    # Perform a left outer join based on the 'Date' column
    merged_df = pd.merge(df_price, df_news, on='Date', how='left')

    # Fill NaN values with 0
    merged_df.fillna(0, inplace=True)

    # Drop duplicate rows
    merged_df.drop_duplicates(inplace=True)

    # Reset the index
    merged_df.reset_index(drop=True, inplace=True)

    # Convert 'sentiment_score' column to integer
    merged_df['sentiment_score'] = merged_df['sentiment_score'].astype(int)

    return merged_df

stock_symbols = ['AAPL']  # Add more stock symbols as needed
tk = 'AAPL'
processed_df = preprocess_stock_data(tk)

processed_df.head(20)


,Date,Open,High,Low,Close,Adj Close,Volume,title,summary,average_overall_sentiment_score,average_relevance_score,average_ticker_sentiment_score,sentiment_score
0,2022-03-01,164.699997,166.600006,161.970001,163.199997,161.774796,83474400,"US stocks fall, oil tops $105 as Ukraine crisi...",A surge in oil sent shivers through risky asse...,-0.277460,0.062711,-0.247750,0
1,2022-03-02,164.389999,167.360001,162.949997,166.559998,165.105453,79724800,"Rich Russians turn to luxury jewellery, watche...",With sanctions on Russia sending the ruble plu...,-0.118323,0.051474,0.101278,0
2,2022-03-03,168.470001,168.910004,165.550003,166.229996,164.778336,76678400,"Market Rallies, But Still Must Do This; 5 Stoc...",Dow Jones Futures: Stock Market Rallies On 'Ni...,-0.207306,0.157168,-0.104088,0
3,2022-03-04,164.490005,165.550003,162.100006,163.169998,161.745056,83737200,Here's How Much a $1000 Investment in Cirrus L...,Holding on to popular or trending stocks for t...,0.147811,0.181502,0.060465,1
4,2022-03-07,163.360001,165.020004,159.039993,159.300003,157.908875,96418800,Apple Co-Founder Says Many Cryptocurrencies Ar...,Apple Inc. ( NASDAQ: AAPL ) co-founder Steve...,0.104481,0.332836,0.165065,1
5,2022-03-08,158.820007,162.880005,155.800003,157.440002,156.065125,131148300,This Potentially Revolutionary Green Energy St...,Bloom Energy is now reaching scale and about t...,0.071402,0.384109,0.053857,1
6,2022-03-09,161.479996,163.410004,159.410004,162.949997,161.526993,91454900,Why Apple Stock Lagged the Market Today Apple'...,Investors didn't seem overly impressed by the ...,0.070802,0.400378,0.051578,1
7,2022-03-10,160.199997,160.389999,155.979996,158.520004,157.135681,105342000,Here's What Social Media Had to Say About Appl...,From the brief Spotify outage to a Big Mac ref...,-0.084905,0.396548,-0.120464,0
8,2022-03-11,158.929993,159.279999,154.500000,154.729996,153.378754,96970100,Apple CEO Tim Cook Comes Out Publicly Against ...,Apple Inc ( NASDAQ: AAPL ) CEO Tim Cook said...,0.078167,0.371532,0.084833,1
9,2022-03-14,151.449997,154.119995,150.100006,150.619995,149.304672,108732100,Rally Attempt Back Near Lows; 3 IBD 50 Stocks ...,Dow Jones Futures: Market Rally Nears Lows Ami...,-0.148062,0.402060,-0.176531,0
